# import def func

In [1]:
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

from JohnsonUtil import LoggerFactory
log = LoggerFactory.log
# log_level = LoggerFactory.DEBUG
# log_level = LoggerFactory.INFO
log_level = LoggerFactory.ERROR
log.setLevel(log_level)
# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

log = LoggerFactory.log

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        #200
        dl=ct.duration_date_week
    else:
        #300
        dl=ct.duration_date_month
    filter='y'
#     import pdb;pdb.set_trace()
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        print(eval(f'lastpTDX_DF_{resample}.shape'))
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-27',
 code
 833429    康比特
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/29/25 23:06:32'

code
603083   2025-06-29 15:00:01
300374   2025-06-29 15:35:30
Name: ticktime, dtype: datetime64[ns]

451.0451525290807

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
[]
G:\sina_MultiIndex_data.h5


#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [10]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [11]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [12]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [13]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5183 it:0.3 lastpTDX_DF_3d not in locals
fname: tdx_last_df keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-06-29 23:06:35,115] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4442): tdx Out:3 code:['601028', '600705']


tdxdataT:2.04
DLTDXE:2.04 

In [14]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:980 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300993,玉马科技,20.03,35,2025-04-09,14.68,11.76,8.19,55.41,8.19,1.20,-0.1
300991,创益通,20.02,17,2025-04-09,28.97,24.19,13.57,55.69,13.57,0.00,-0.1
300414,中光防雷,20.00,228,2025-04-09,12.90,8.76,5.87,65.31,5.87,0.10,0.1
301205,联特科技,17.36,9,2025-04-09,101.00,84.19,48.97,45.81,48.97,0.10,1.1
835184,国源科技,16.50,0,2025-04-25,19.70,15.33,10.33,50.31,13.31,0.00,0.0
301357,北方长龙,15.00,6,2025-04-25,80.50,60.00,26.45,58.09,31.81,1.10,18.3
300591,万里马,12.30,491,2025-04-09,11.23,6.40,3.42,59.59,3.42,35.31,0.1
300501,海顺新材,11.89,30,2025-04-09,18.72,15.55,12.03,49.19,12.03,1.30,-0.1
838971,天马新材,11.81,0,2025-04-09,40.81,33.88,21.80,1.20,21.80,1.30,0.0


In [15]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.23
boll             4
ral             11
fib              0
percmax       19.0
macd          0.41
macddif       0.87
macddea       0.66
macdlast2     0.42
macdlast3     0.34
lasth1d       18.3
lastl1d      16.73
lastl2d      16.16
lastl3d      16.12
lastp1d      16.73
ma51d        16.33
ma52d        15.97
ma53d        15.51
ma54d        15.17
Name: 301038, dtype: object

In [16]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [17]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5183 it:0.3 lastpTDX_DF_w not in locals
fname: tdx_last_df keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [18]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:365 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300650,太龙股份,20.02,43,2025-01-10,17.03,16.41,10.25,0.10,10.28,-1.10,-1.10
300414,中光防雷,20.00,228,2025-04-11,12.90,9.01,5.87,78.88,5.87,1.20,0.10
301024,霍普股份,19.97,7,2025-01-27,47.88,45.56,20.06,74.88,20.89,-0.10,67.88
301205,联特科技,17.36,9,2025-04-11,101.00,88.75,48.97,56.69,48.97,1.30,-0.10
301357,北方长龙,15.00,6,2025-01-27,80.50,39.66,26.45,85.19,28.48,30.09,1.20
920106,林泰新材,10.78,0,2025-01-17,113.00,106.00,37.22,1.30,37.62,-0.10,-1.10
870357,雅葆轩,10.18,0,2025-01-03,29.23,28.69,15.74,37.91,17.88,-0.90,0.00
603616,韩建河山,10.07,54,2025-04-11,5.03,4.54,2.76,33.09,2.76,1.20,1.20
002402,和而泰,10.05,208,2025-01-03,23.33,22.77,14.98,44.09,16.38,-1.10,1.20


In [19]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

### resample M

In [20]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5183 it:0.3 lastpTDX_DF_m not in locals
fname: tdx_last_df keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [21]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:2022 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300993,玉马科技,20.03,35,2025-04-30,14.68,11.19,7.59,66.81,8.19,-1.1,-0.90
300991,创益通,20.02,17,2024-09-30,28.97,20.66,12.60,56.41,12.60,0.0,1.20
300414,中光防雷,20.00,228,2025-04-30,12.90,8.71,5.66,82.38,5.87,0.0,35.09
301024,霍普股份,19.97,7,2025-01-27,47.88,29.08,17.23,57.91,20.06,0.0,1.20
301408,华人健康,19.97,81,2025-01-27,14.54,12.25,9.20,45.91,10.03,-1.1,21.09
301357,北方长龙,15.00,6,2025-01-27,80.50,40.41,22.41,134.50,28.11,0.0,1.10
300591,万里马,12.30,491,2025-01-27,11.23,5.81,2.67,127.31,3.56,0.0,31.80
301315,威士顿,12.03,14,2024-10-31,58.50,67.88,27.56,1.30,39.88,-1.1,0.00
838971,天马新材,11.81,0,2024-09-30,40.81,42.59,7.60,41.19,7.60,-1.1,48.59


In [22]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d     18.3
lastl1d    13.92
lastl2d    13.39
lastl3d    11.53
lastp1d    16.73
ma51d       14.6
ma52d      13.97
ma53d      13.92
ma54d      14.48
Name: 301038, dtype: object

### resample D

In [23]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx :0 b1>:5183 it:0.3 lastpTDX_DF_d not in locals
fname: tdx_last_df keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-06-29 23:06:49,829] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4442): tdx Out:3 code:['601028', '600705']


tdxdataT:1.97
DLTDXE:1.97 

(5179, 218)

In [24]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
# if len(td_code) < 30:
#     write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:149 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300993,玉马科技,20.03,35,2025-04-07,14.68,11.96,10.35,47.59,8.50,1.30,1.3
300591,万里马,12.30,491,2025-04-07,11.23,7.02,5.43,45.00,3.68,35.31,1.3
002205,国统股份,10.02,18,2025-04-08,11.75,10.57,9.62,42.69,8.03,1.30,0.0
600152,维科技术,10.02,167,2025-04-08,6.92,6.52,5.71,32.31,5.03,1.30,1.2
600967,内蒙一机,10.01,372,2025-04-07,17.59,14.78,11.31,32.09,9.86,30.50,1.1
600743,华远控股,10.00,105,2025-04-07,2.31,2.27,1.80,32.31,1.59,1.20,0.0
001236,弘业期货,9.99,5,2025-04-08,14.53,11.09,9.04,30.30,8.09,1.30,0.0
600960,渤海汽车,9.94,22,2025-04-08,5.09,5.05,3.51,1.30,2.99,0.10,0.0
871634,新威凌,8.96,0,2025-04-07,31.14,31.52,24.30,29.30,20.09,0.10,1.3


In [25]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

#### filter test

In [26]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=5)

Count:209 216


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
300414,中光防雷,20.00,228,2025-04-09,12.90,8.76,5.87,65.31,5.87,0.1,0.10
301357,北方长龙,15.00,6,2025-04-25,80.50,60.00,26.45,58.09,31.81,1.1,18.30
870357,雅葆轩,10.18,0,2025-04-09,29.23,25.38,15.74,38.19,15.74,0.0,0.00
603616,韩建河山,10.07,54,2025-04-09,5.03,4.46,2.76,38.00,2.76,1.2,0.00
002104,恒宝股份,10.05,674,2025-05-23,19.16,14.71,6.45,38.31,7.05,1.1,33.41


In [27]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [28]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:2022
tw_code:365
t3d_code:980
td_code:149
Count:26 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
600967,内蒙一机,10.01,372,2025-04-07,17.59,14.78,11.31,32.09,9.86,30.5,1.1
688061,灿瑞科技,8.18,0,2025-04-07,33.06,30.69,28.58,36.50,24.03,1.3,0.0
603758,秦安股份,7.85,29,2025-04-08,16.48,15.30,12.11,43.59,9.76,1.3,7.3
600331,宏达股份,7.65,186,2025-04-08,8.58,7.91,6.93,36.09,5.95,1.2,0.0
601606,长城军工,7.58,295,2025-04-07,26.41,20.30,12.89,31.09,10.79,17.3,17.3
688498,源杰科技,6.81,0,2025-04-07,178.38,170.25,127.25,32.81,92.00,0.0,0.0
600478,科力远,5.76,483,2025-04-08,6.24,5.97,4.65,32.31,4.20,0.1,11.3
300101,振芯科技,5.12,41,2025-04-07,22.36,21.20,16.69,39.81,14.34,1.2,0.0
003026,中晶科技,5.11,2,2025-04-08,36.59,35.44,30.20,28.80,24.20,0.1,14.1


#### check data

In [29]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

26

In [30]:
show_tdx_data(td_code.loc[code_f])

Count:26 218


,name,percent,couts,date,close,hmax,lmin,perc1d,llow,perc3d,perc5d
code,,,,,,,,,,,
600967,内蒙一机,10.01,372,2025-04-07,17.59,14.78,11.31,32.09,9.86,30.50,1.1
688061,灿瑞科技,8.18,0,2025-04-07,33.06,30.69,28.58,36.50,24.03,1.30,0.0
603758,秦安股份,7.85,29,2025-04-08,16.48,15.30,12.11,43.59,9.76,1.30,7.3
600331,宏达股份,7.65,186,2025-04-08,8.58,7.91,6.93,36.09,5.95,1.20,0.0
601606,长城军工,7.58,295,2025-04-07,26.41,20.30,12.89,31.09,10.79,17.30,17.3
688498,源杰科技,6.81,0,2025-04-07,178.38,170.25,127.25,32.81,92.00,0.00,0.0
600478,科力远,5.76,483,2025-04-08,6.24,5.97,4.65,32.31,4.20,0.10,11.3
300101,振芯科技,5.12,41,2025-04-07,22.36,21.20,16.69,39.81,14.34,1.20,0.0
003026,中晶科技,5.11,2,2025-04-08,36.59,35.44,30.20,28.80,24.20,0.10,14.1


### Write toblk

In [32]:
# write_code_to_blk(code_f,'077',rewrite=False)

write blk [Y] or [N]: y
Rewrite code [Y] or append [N](defalut:N): b


all write to D:\MacTools\WinTools\new_tdx2\T0002\blocknew\077.blk:23
all write to D:\MacTools\WinTools\zd_dxzq\T0002\blocknew\077.blk:22
write:26 block_path:D:\MacTools\WinTools\new_tdx2\T0002\blocknew\077.blk
